# Continuous functions minimization by dynamic random search technique (DRASET)

This works presents an algorithm DRASET for continuous functions. DRASET was first introduced in this article: https://www.sciencedirect.com/science/article/pii/S0307904X06002071 All steps of DRASET can be found there. The main ideas of DRASET are:
* there are two stages of the algorithm: the general search phase and the local search phase
* in the general search phase the algorithm tries to search as much space as possible and remembers the best found solution
* in the local search phase algorithm tries to improve the best solution found in the general search phase
* in the both phases the algorithm works with the best found solution and the current solution
* a random vector generated from a range $\langle-\alpha, \alpha \rangle$ is added or substracted from the current solution and the function is evaluated at this number
* then there are some decisions, for more details see the article
* important is that if a better solution than the current best solution is found, the current solution doesn't change - it prevents from stucking in a local optimum
* in the general search phase alpha differs every $N_{general}$ steps and in the local search phase alpha differs every $N_{local}$ steps
* $E$ (epochs) represents the number of generating the random vector
* the general search phase lasts $E - 2*N_{general}$ steps
* the local search phase lasts $2*N_{general}$ steps
* the number of epochs doesn't equal the number of function evaluations, in the worst case the function can be evaluated for $2*E$ times

In [1]:
# Import path to source directory (bit of a hack in Jupyter)
import sys
import os
pwd = %pwd
sys.path.append(os.path.join(pwd, os.path.join('..', 'src')))

# Ensure modules are reloaded on any change (very useful when developing code on the fly)
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from objfun_plato import Plato
from objfun_banana import Banana
from heur_draset import Draset
from heur_aux import Correction
from heur_sg import ShootAndGo

In [3]:
np.random.seed(6)

We will study DRASET's behavior on two functions - Rastrigin's function 6 and Rosenbrock's valley (De Jong's function 2). Both functions are defined here: http://www.geatbx.com/docu/fcnindex-01.html There are also graphs of these functions.

Random shooting is chosen as a baseline method. 

We start with Rastrigin's function 6. This function has many local minima.

In [4]:
function = Plato(n = 2, eps = 0.1)
corr = Correction(function)  # correction is not discussed in the original article, I choose the simpliest one

In [5]:
RUNS = 500
MAXEV = 50000  # the heuristic is implemented with maxeval parameter, I don't want to restrict heuristic by this parameter, so I set it at this high level

We define two experiments: DRASET and random shooting.

In [6]:
def experiment_draset(of, maxeval, num_runs, E, alfa0, N_general, N_local, correction): 
    results = []
    for i in tqdm(range(num_runs), 'Testing E={}, alfa0={}, N_general={}, N_local={}'.format(E, alfa0, N_general, N_local)):
        result = Draset(of, maxeval, E, alfa0, N_general, N_local, correction).search()
        result['run'] = i
        result['heur'] = 'Draset_{}_{}_{}_{}'.format(E, alfa0, N_general, N_local)
        result['E'] = E
        result['alfa0'] = alfa0
        result['N_general'] = N_general
        result['N_local'] = N_local
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'E', 'alfa0', 'N_general', 'N_local', 'best_x', 'best_y', 'neval', 'epoch', 'alfa', 'f_x', 'log_data'])

In [7]:
def experiment_random(of, maxeval, num_runs):
    results = []
    for i in tqdm(range(num_runs), 'Testing maxeval={}'.format(maxeval)):
        result = ShootAndGo(of, maxeval, hmax = 0).search()
        result['run'] = i
        result['heur'] = 'Random_{}'.format(maxeval)
        result['maxeval'] = maxeval
        results.append(result)
    return pd.DataFrame(results, columns=['heur', 'run', 'maxeval', 'best_x', 'best_y', 'neval'])

We define statistics for a later analysis.

In [8]:
def rel(x):
    return len([n for n in x if n < np.inf])/len(x)

def mne(x):
    return np.mean([n for n in x if n < np.inf])

def feo(x):
    return mne(x)/rel(x)

In [9]:
def mean(x):
    return np.mean(x)

def med(x):
    return np.median(x)

Now we run the first experiment. In the article initial $\alpha = 1$ and $N_{local} = \frac{1}{5} N_{general}$ setting was recomended.

In [10]:
table_draset = pd.DataFrame()

for E in [250, 500, 1000, 2000, 5000]:
    for alfa0 in [1]:
        for N_general in [50, 100, 500, 1000, 2000]:
            if 2*N_general < E:  # we will run only settings which make sense
                N_local = N_general/5
                res = experiment_draset(of = function, maxeval = MAXEV, num_runs = RUNS, E = E, alfa0 = alfa0, N_general = N_general, N_local = N_local, correction = corr)
                table_draset = pd.concat([table_draset, res], axis = 0)
    

In [11]:
stats_draset = table_draset.pivot_table(
    index=['heur', 'E', 'alfa0', 'N_general', 'N_local'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_draset = stats_draset.reset_index()

In [12]:
stats_draset.sort_values(by=['feo'])

,heur,E,alfa0,N_general,N_local,feo,mne,rel
2,Draset_2000_1_100_20.0,2000,1,100,20.0,2917.751736,1960.729167,0.672
4,Draset_2000_1_50_10.0,2000,1,50,10.0,2991.960834,2016.581602,0.674
3,Draset_2000_1_500_100.0,2000,1,500,100.0,3192.243864,2049.420561,0.642
7,Draset_5000_1_1000_200.0,5000,1,1000,200.0,3218.379263,3063.897059,0.952
8,Draset_5000_1_100_20.0,5000,1,100,20.0,3238.302571,3056.957627,0.944
11,Draset_5000_1_50_10.0,5000,1,50,10.0,3251.261839,2900.125561,0.892
1,Draset_1000_1_50_10.0,1000,1,50,10.0,3337.473173,1188.140449,0.356
9,Draset_5000_1_2000_400.0,5000,1,2000,400.0,3378.094926,3188.921610,0.944
10,Draset_5000_1_500_100.0,5000,1,500,100.0,3461.255963,3225.890558,0.932
0,Draset_1000_1_100_20.0,1000,1,100,20.0,3631.895050,1205.789157,0.332


The best Feoktistov criterion is obtained for $E = 2000$. 

In [13]:
stats_draset.sort_values(by=['rel'], ascending = False)

,heur,E,alfa0,N_general,N_local,feo,mne,rel
7,Draset_5000_1_1000_200.0,5000,1,1000,200.0,3218.379263,3063.897059,0.952
8,Draset_5000_1_100_20.0,5000,1,100,20.0,3238.302571,3056.957627,0.944
9,Draset_5000_1_2000_400.0,5000,1,2000,400.0,3378.094926,3188.921610,0.944
10,Draset_5000_1_500_100.0,5000,1,500,100.0,3461.255963,3225.890558,0.932
11,Draset_5000_1_50_10.0,5000,1,50,10.0,3251.261839,2900.125561,0.892
4,Draset_2000_1_50_10.0,2000,1,50,10.0,2991.960834,2016.581602,0.674
2,Draset_2000_1_100_20.0,2000,1,100,20.0,2917.751736,1960.729167,0.672
3,Draset_2000_1_500_100.0,2000,1,500,100.0,3192.243864,2049.420561,0.642
1,Draset_1000_1_50_10.0,1000,1,50,10.0,3337.473173,1188.140449,0.356
0,Draset_1000_1_100_20.0,1000,1,100,20.0,3631.895050,1205.789157,0.332


We can see a clear pattern that for more epochs we get higher reliability. But generally we can't say anything about a relation between the number of epochs and $N_{general}$ and better or worse reliability or Feoktistov criterion.

Now we focus on the best found value of the function after given number of epochs. The optimum equals 0.

In [14]:
stats_draset_best = table_draset.pivot_table(
    index=['heur', 'E', 'alfa0', 'N_general', 'N_local'],
    values=['best_y'],
    aggfunc=(mean, med)
)['best_y']
stats_draset_best = stats_draset_best.reset_index()

In [15]:
stats_draset_best.sort_values(by=['med'])

,heur,E,alfa0,N_general,N_local,mean,med
9,Draset_5000_1_2000_400.0,5000,1,2000,400.0,0.629077,0.051115
7,Draset_5000_1_1000_200.0,5000,1,1000,200.0,0.457833,0.054198
10,Draset_5000_1_500_100.0,5000,1,500,100.0,0.750341,0.054203
8,Draset_5000_1_100_20.0,5000,1,100,20.0,0.601929,0.056660
11,Draset_5000_1_50_10.0,5000,1,50,10.0,1.053386,0.058485
2,Draset_2000_1_100_20.0,2000,1,100,20.0,0.568706,0.073588
4,Draset_2000_1_50_10.0,2000,1,50,10.0,0.926227,0.075878
3,Draset_2000_1_500_100.0,2000,1,500,100.0,0.745639,0.079698
1,Draset_1000_1_50_10.0,1000,1,50,10.0,1.527605,0.176313
0,Draset_1000_1_100_20.0,1000,1,100,20.0,1.369208,0.194272


The table is sorted by median because it is a more robust statistic. We can see that with more epochs the algorithm finds a solution closer to the optimum.

Now we run the random shooting algorithm. We set a maximum number of evaluations of the function as $2*E$ because, as it was said before, in the worst case during DRASET the function can be evaluated $2*E$ times.

In [16]:
table_random = pd.DataFrame()

for maxeval in [500, 1000, 2000, 4000, 10000]:
    res = experiment_random(of = function, maxeval = maxeval, num_runs = RUNS)
    table_random = pd.concat([table_random, res], axis = 0)

In [17]:
stats_random = table_random.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_random = stats_random.reset_index()

In [18]:
stats_random.sort_values(by=['rel'], ascending = False)

,heur,feo,mne,rel
1,Random_10000,34292.275383,5075.256757,0.148
3,Random_4000,30945.000000,1856.700000,0.060
0,Random_1000,20396.449704,530.307692,0.026
2,Random_2000,45627.218935,1186.307692,0.026
4,Random_500,11625.000000,46.500000,0.004


It is obvious that we get much worse results than by DRASET. Both Feoktistov criterion and reliability are worse.

In [19]:
stats_random_best = table_random.pivot_table(
    index=['heur'],
    values=['best_y'],
    aggfunc=(mean, med)
)['best_y']
stats_random_best = stats_random_best.reset_index()

In [20]:
stats_random_best.sort_values(by=['med'])

,heur,mean,med
1,Random_10000,0.549803,0.460388
3,Random_4000,0.913500,1.019392
2,Random_2000,1.311941,1.238497
0,Random_1000,1.725059,1.617650
4,Random_500,2.629286,2.418444


The best found solution is also worse than the solution found by DRASET.

Now we analyze the second function: Rosenbrock's valley (De Jong's function 2). The global optimum is inside a long, narrow, parabolic shaped flat valley and equals 0.

In [21]:
function = Banana(n = 2, eps = 0.01)  # In this case we want to find more precise solution.
corr = Correction(function)  # correction is not discussed in the original article, I choose the simpliest one

First we run DRASET experiment.

In [22]:
table_draset2 = pd.DataFrame()

for E in [250, 500, 1000, 2000, 5000]:
    for alfa0 in [1]:
        for N_general in [50, 100, 500, 1000, 2000]:
            if 2*N_general < E:  # we will run only settings which make sense
                N_local = N_general/5
                res = experiment_draset(of = function, maxeval = MAXEV, num_runs = RUNS, E = E, alfa0 = alfa0, N_general = N_general, N_local = N_local, correction = corr)
                table_draset2 = pd.concat([table_draset2, res], axis = 0)
    

In [23]:
stats_draset2 = table_draset2.pivot_table(
    index=['heur', 'E', 'alfa0', 'N_general', 'N_local'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_draset2 = stats_draset2.reset_index()

In [24]:
stats_draset2.sort_values(by=['feo'])

,heur,E,alfa0,N_general,N_local,feo,mne,rel
5,Draset_250_1_100_20.0,250,1,100,20.0,903.367347,252.942857,0.280
13,Draset_500_1_50_10.0,500,1,50,10.0,959.502155,449.047009,0.468
12,Draset_500_1_100_20.0,500,1,100,20.0,1047.047469,483.735931,0.462
1,Draset_1000_1_50_10.0,1000,1,50,10.0,1086.066749,790.656593,0.728
6,Draset_250_1_50_10.0,250,1,50,10.0,1097.064752,276.460317,0.252
0,Draset_1000_1_100_20.0,1000,1,100,20.0,1110.723069,830.820856,0.748
3,Draset_2000_1_500_100.0,2000,1,500,100.0,1309.313607,1201.949891,0.918
4,Draset_2000_1_50_10.0,2000,1,50,10.0,1322.176390,1192.603104,0.902
2,Draset_2000_1_100_20.0,2000,1,100,20.0,1380.255255,1225.666667,0.888
9,Draset_5000_1_2000_400.0,5000,1,2000,400.0,1515.075846,1487.804481,0.982


Acording to Feoktistov criterion 250 epochs and $N_{general}$ = 100 is the best setting for DRASET. But reliability of this setting isn't high. For $E = 2000$ or $5000$ Feoktistov criterion is worse than for less epochs because of more function evaluations.

In [25]:
stats_draset2.sort_values(by=['rel'], ascending = False)

,heur,E,alfa0,N_general,N_local,feo,mne,rel
9,Draset_5000_1_2000_400.0,5000,1,2000,400.0,1515.075846,1487.804481,0.982
7,Draset_5000_1_1000_200.0,5000,1,1000,200.0,1577.939508,1533.757202,0.972
10,Draset_5000_1_500_100.0,5000,1,500,100.0,1520.982574,1478.395062,0.972
8,Draset_5000_1_100_20.0,5000,1,100,20.0,1544.440104,1482.662500,0.960
11,Draset_5000_1_50_10.0,5000,1,50,10.0,1524.166205,1447.957895,0.950
3,Draset_2000_1_500_100.0,2000,1,500,100.0,1309.313607,1201.949891,0.918
4,Draset_2000_1_50_10.0,2000,1,50,10.0,1322.176390,1192.603104,0.902
2,Draset_2000_1_100_20.0,2000,1,100,20.0,1380.255255,1225.666667,0.888
0,Draset_1000_1_100_20.0,1000,1,100,20.0,1110.723069,830.820856,0.748
1,Draset_1000_1_50_10.0,1000,1,50,10.0,1086.066749,790.656593,0.728


We can see a clear pattern that for more epochs we get a higher reliability. 

In [26]:
stats_draset_best2 = table_draset2.pivot_table(
    index=['heur', 'E', 'alfa0', 'N_general', 'N_local'],
    values=['best_y'],
    aggfunc=(mean, med)
)['best_y']
stats_draset_best2 = stats_draset_best2.reset_index()

In [27]:
stats_draset_best2.sort_values(by=['med'])

,heur,E,alfa0,N_general,N_local,mean,med
9,Draset_5000_1_2000_400.0,5000,1,2000,400.0,3.463811,0.004823
8,Draset_5000_1_100_20.0,5000,1,100,20.0,0.896391,0.004917
7,Draset_5000_1_1000_200.0,5000,1,1000,200.0,1.758937,0.004998
11,Draset_5000_1_50_10.0,5000,1,50,10.0,1.721629,0.005208
3,Draset_2000_1_500_100.0,2000,1,500,100.0,3.587058,0.005298
10,Draset_5000_1_500_100.0,5000,1,500,100.0,2.868470,0.005311
4,Draset_2000_1_50_10.0,2000,1,50,10.0,0.979411,0.005473
2,Draset_2000_1_100_20.0,2000,1,100,20.0,1.739398,0.005737
1,Draset_1000_1_50_10.0,1000,1,50,10.0,2.688675,0.006782
0,Draset_1000_1_100_20.0,1000,1,100,20.0,5.250113,0.006909


The table is sorted by median because it is a more robust statistic. We can see that with more epochs (only with one exception) the algorithm finds a solution closer to the optimum.

Now we run the random shooting algorithm. We set a maximum number of evaluations of the function as $2*E$ because, as it was said before, in the worst case during DRASET the function can be evaluated $2*E$ times.

In [28]:
table_random2 = pd.DataFrame()

for maxeval in [500, 1000, 2000, 4000, 10000]:
    res = experiment_random(of = function, maxeval = maxeval, num_runs = RUNS)
    table_random2 = pd.concat([table_random2, res], axis = 0)

In [29]:
stats_random2 = table_random2.pivot_table(
    index=['heur'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
stats_random2 = stats_random2.reset_index()

In [30]:
stats_random2.sort_values(by=['rel'], ascending = False)

,heur,feo,mne,rel
1,Random_10000,4027.983369,3359.338129,0.834
3,Random_4000,3185.595568,1694.736842,0.532
2,Random_2000,3364.891958,901.791045,0.268
0,Random_1000,2686.867936,440.646341,0.164
4,Random_500,3376.859012,276.902439,0.082


In [31]:
stats_random_best2 = table_random2.pivot_table(
    index=['heur'],
    values=['best_y'],
    aggfunc=(mean, med)
)['best_y']
stats_random_best2 = stats_random_best2.reset_index()

In [32]:
stats_random_best2.sort_values(by=['med'])

,heur,mean,med
1,Random_10000,0.006341,0.005338
3,Random_4000,0.013110,0.009418
2,Random_2000,0.027706,0.020287
0,Random_1000,0.057218,0.039152
4,Random_500,0.110493,0.073760


We again get worse results than by DRASET. 

## Conclusion
If we compare random shooting and DRASET, it is obvious that DRASET is a better algorithm than random shooting. DRASET has lower Feoktistov criterion, a higher reliability and the best found solution is closer to the optimum. If we care about the reliability or finding the closest solution, I recommend to run thousends of epochs. If we care only about Feoktistov criterion, it is difficult to recommend the optimal setting. Probably it would be less epochs than for the best reliability. I have no recommendations about $N_{general}$. There is no visible influence of it's value. Number of epochs is a more important parameter.